In [ ]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")
import numpy as np
import matplotlib.pyplot as plt

import pickle

In [ ]:
file_2 = "/vol/bitbucket/ad6013/Research/gp-causal/results/fullscore-sim-2-gplvm-sqexp-reinit20.p"


with open(file_2, "rb") as f:
    results_2 = pickle.load(f)

In [ ]:
file_1 = "/vol/bitbucket/ad6013/Research/gp-causal/results/fullscore-sim-gplvm-sqexp-reinit20.p"


with open(file_1, "rb") as f:
    results_1 = pickle.load(f)

In [ ]:
# log odds will be causal hypothesis over anticausal
scores_1 = {}
for idx, i in enumerate(results_1['scores']):
    if idx > 49:
        break
    causal = i[0]
    anti_causal = i[1]
    scores_1[idx] = (causal, anti_causal)
    if sum(causal) > sum(anti_causal):
        print(idx)

In [ ]:
# log odds will be causal hypothesis over anticausal
scores_2 = {}
for idx, i in enumerate(results_2[1]):
    idx += 50
    causal = i[0]
    anti_causal = i[1]
    scores_2[idx] = (causal, anti_causal)
    if sum(causal) > sum(anti_causal):
        print(idx)

In [ ]:
file_3 = "/vol/bitbucket/ad6013/Research/gp-causal/results/fullscore-sim-gplvm-sqexp-reinit5-numind110.p"


with open(file_3, "rb") as f:
    results_3 = pickle.load(f)

In [ ]:
wrong = [1,
    8,
    9,
    10,
    11,
    13,
    15,
    18,
    23,
    25,
    31,
    33,
    38,
    39,
    46,
    48,
    55,
    58,
    63,
    71,
    74,
    76,
    84,
    87]
scores_3 = {}
for idx, i in enumerate(results_3['scores']):
    causal = i[0]
    anti_causal = i[1]
    scores_3[wrong[idx]] = (causal, anti_causal)

In [ ]:
if 50 in scores_2.keys():
    print(1)

In [ ]:
final_scores = {}
all_log_diffs = {}
correct_idx = []
wrong_idx = []
for i in range(100):
    if i in scores_3.keys():
        causal_x_3 = scores_3[i][0][0]
        causal_y_x_3 = scores_3[i][0][1]
        anticausal_y_3 = scores_3[i][1][0]
        anticausal_x_y_3 = scores_3[i][1][1]
        compare = True
    else:
        compare = False 
    if i in scores_1.keys():
        causal_x = scores_1[i][0][0]
        causal_y_x = scores_1[i][0][1]
        anticausal_y = scores_1[i][1][0]
        anticausal_x_y = scores_1[i][1][1]
    elif i in scores_2.keys():
        causal_x = scores_2[i][0][0]
        causal_y_x = scores_2[i][0][1]
        anticausal_y = scores_2[i][1][0]
        anticausal_x_y = scores_2[i][1][1]
    else:
        raise ValueError(f"{i} not found in results!")
    if compare:
        final_causal_x = min(causal_x, causal_x_3)
        final_causal_y_x = min(causal_y_x, causal_y_x_3)
        final_anticausal_y = min(anticausal_y, anticausal_y_3)
        final_anticausal_x_y = min(anticausal_x_y, anticausal_x_y_3)
    else:
        final_causal_x = causal_x
        final_causal_y_x = causal_y_x
        final_anticausal_y = anticausal_y
        final_anticausal_x_y = anticausal_x_y
    final_scores[i] = (
        (final_causal_x, final_causal_y_x),
        (final_anticausal_y, final_anticausal_x_y)
    )
    score_causal = final_causal_x + final_causal_y_x
    score_anticausal = final_anticausal_y + final_anticausal_x_y
    log_diff = score_causal - score_anticausal
    all_log_diffs[i] = log_diff
    if log_diff < 0:
        correct_idx.append(i)
    elif log_diff > 0:
        wrong_idx.append(i)
    else:
        print(
            f"Undecided: {i}"
        )


In [ ]:
wrong_idx

In [ ]:
plt.bar(np.arange(len(all_log_diffs)), list(all_log_diffs.values()))
plt.ylabel("Log difference")
plt.xlabel("Dataset Index")
# plt.ylim(-400, 200)
plt.show()

In [ ]:
# Calculate ROC curve and AUC 
# The score value should be - causal + anticausal
# This is because a higher score should mean causal and a lower score mean anticausal

# We also need to ensure half the labels are anticausal and half the labels are causal
# We thus need to generate indexes that will be the anticausal direction is true
# then for the other ones, we will multiply the scores by -1

anticausal_labels = np.random.choice(100, 50, replace=False)
causal_labels = np.array(
    list(
        set(np.arange(100)) - set(anticausal_labels)
    )
)
y_labels = np.zeros(100)
y_scores = np.zeros(100)
for i in range(100):
    if i in anticausal_labels:
        y_labels[i] = -1
        y_scores[i] = all_log_diffs[i]
    elif i in causal_labels:
        y_labels[i] = 1
        y_scores[i] = -1 * all_log_diffs[i]
    else:
        raise ValueError(f"Missed {i}")
    

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc


fpr, tpr, _ = roc_curve(y_labels, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

In [ ]:
textstr = 'roc auc=%.3f\n'%(
    roc_auc
)
plt.text(0.8, 0.2, textstr, fontsize=10)
plt.plot(
    fpr, tpr
)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.show()

In [ ]:
file = "/vol/bitbucket/ad6013/Research/gp-causal/results/fullscore-cep-gplvm-sqexp-reinit20.p"


with open(file, "rb") as f:
    results_tub = pickle.load(f)

In [ ]:
corr, wrong, weight = results_tub["correct_idx"], results_tub["wrong_idx"], results_tub["weight"]
correct_weight = [weight[i] for i in corr]
wrong_weight = [weight[i] for i in wrong]
accuracy = np.sum(correct_weight) / (np.sum(correct_weight) + np.sum(wrong_weight))
print(f"\n Final accuracy: {accuracy}")

In [ ]:
# log odds will be causal hypothesis over anticausal
log_odds_tub = []
for i in results_tub['scores']:
    causal = sum(i[0])
    anti_causal = sum(i[1])
    log_odds_tub.append(causal - anti_causal)

In [ ]:
plt.bar(np.arange(len(log_odds_tub)), log_odds_tub)
plt.ylabel("Log odds - 1")
plt.xlabel("Dataset Index")
plt.ylim(-1000, 1000)
plt.show()

In [ ]:
for idx, i in enumerate(log_odds_tub):
    print(
        f"{idx}, {i}"
    )


In [ ]:
Correct: 40, Wrong: 17
Correct: 42, Wrong: 8

In [ ]:
wrong_idx

In [ ]:
# Plot the results that we got wrong
from data.get_data import get_simulated_pairs_dataset


x, y, weight = get_simulated_pairs_dataset(
    data_path='/vol/bitbucket/ad6013/Research/gp-causal/data/sim_pairs/files'
)

for i in range(100):
    if i in wrong_idx:
        input, output = x[i], y[i]
        input = (input - input.mean(axis=0)[:, None]) / input.std(axis=0)[:, None]
        output = (output - output.mean(axis=0)[:, None]) / output.std(axis=0)[:, None]
        print(f"\n {i}: {all_log_diffs[i]}")
        plt.scatter(input[:, 0], output[:, 0])
        plt.show()

# Cha pairs results analysis

In [ ]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")

In [ ]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np


work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
file = "fullscore-cha_pairs-gplvm-reinit20-numind200_start:0_end:150.p"
file_2 = "fullscore-cha_pairs-gplvm-reinit20-numind200_start:150_end:300.p"

In [ ]:
with open(f"{work_dir}/results/{file}", "rb") as f:
    results = pickle.load(f)
with open(f"{work_dir}/results/{file_2}", "rb") as f:
    results_2 = pickle.load(f)

In [ ]:
from data.get_data import get_cha_pairs_dataset


x, y, weight, target = get_cha_pairs_dataset(data_path=f"{work_dir}/data/cha_pairs/files")

In [ ]:
num_runs_1 = len(results['scores'])
num_runs_2 = len(results_2['scores'])
all_bfgs_cha_scores = {} 

y_labels = np.zeros(num_runs_1 + num_runs_2)
y_scores = np.zeros(num_runs_1 + num_runs_2)
# causal labels are 1 thus we need to multiply all scores by -1
for i in range(num_runs_1):
    y_label_i = target[i][0]
    y_labels[i] = y_label_i
    # Calculate the score
    # This is a tuple
    scores = results['scores'][i]
    all_bfgs_cha_scores[i] = scores
    causal = sum(scores[0])
    anti_causal = sum(scores[1])
    final_score = - causal + anti_causal
    y_scores[i] = final_score
for i in range(150, 150 + num_runs_2):
    idx = i - 150 + num_runs_1
    y_label_i = target[i][0]
    y_labels[idx] = y_label_i
    # Calculate the score
    # This is a tuple
    scores = results_2['scores'][i - 150]
    all_bfgs_cha_scores[i] = scores
    causal = sum(scores[0])
    anti_causal = sum(scores[1])
    final_score = - causal + anti_causal
    y_scores[idx] = final_score


In [ ]:
all_bfgs_cha_scores

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score


fpr, tpr, _ = roc_curve(y_labels, y_scores)
roc_auc = auc(fpr, tpr)


In [ ]:
roc_auc

In [ ]:
textstr = 'roc auc=%.3f\n'%(
    roc_auc
)
plt.text(0.8, 0.2, textstr, fontsize=10)
plt.plot(
    fpr, tpr
)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.show()

# Gauss pairs results analysis

In [27]:
import sys
import dill
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")

In [28]:
from data import get_data

In [29]:
data_name = "cha"

In [30]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np



work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
file_1 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:0_end:20.p"
file_2 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:20_end:40.p"
file_3 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:40_end:60.p"
file_4 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:60_end:80.p"
file_5 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:80_end:100.p"
file_6 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:100_end:120.p"
file_7 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:120_end:140.p"
file_8 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:140_end:160.p"
file_9 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:160_end:180.p"
file_10 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:180_end:200.p"
file_11 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:200_end:220.p"
file_12 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:220_end:240.p"
file_13 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:240_end:260.p"
file_14 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:260_end:280.p"
file_15 = f"fullscore-{data_name}_pairs-gplvmgeneralised-reinit10-numind200_start:280_end:300.p"

In [31]:
import importlib
from data import get_data


data_get = getattr(get_data, f"get_{data_name}_pairs_dataset")

x, y, weight, target = data_get(data_path=f"{work_dir}/data/{data_name}_pairs/files")

In [34]:
all_results = {}
for file_idx in range(15):
    with open(f"{work_dir}/results/{eval(f'file_{file_idx + 1}')}", "rb") as f:
        results = dill.load(f)
    all_results.update(results["final_scores"])

In [35]:
def get_scores(best_scores):
    total_runs = len(list(best_scores.keys()))
    y_scores = {}
    y_scores_array = np.zeros(total_runs)
    for idx, run_idx in enumerate(list(best_scores.keys())):
        current_run = best_scores[run_idx]
        causal_score = current_run.best_loss_x + current_run.best_loss_y_x
        anticausal_score = current_run.best_loss_y + current_run.best_loss_x_y
        y_scores[run_idx] = - causal_score + anticausal_score
        y_scores_array[idx] = - causal_score + anticausal_score 
    return y_scores, y_scores_array

In [36]:
from utils import return_all_scores, return_best_causal_scores, get_correct


all_x, all_y_x, all_y, all_x_y = return_all_scores(all_results)
best_scores = return_best_causal_scores(all_x, all_y_x, all_y, all_x_y)

In [37]:
_, y_scores = get_scores(best_scores)

In [38]:
sum(target)

array([-10.])

In [49]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score


fpr, tpr, _ = roc_curve(target, y_scores)
roc_auc = auc(fpr, tpr)

# roc_auc = roc_auc_score(target, y_scores)

In [48]:
roc_auc

0.8080533926585095

In [50]:
roc_auc

0.8080533926585095

In [12]:
y_labels = []
y_scores = []
wrong_idx = []
all_bfgs_scores = {}
all_causal_scores = {}

for file_idx in range(15):
    with open(f"{work_dir}/results/{eval(f'file_{file_idx + 1}')}", "rb") as f:
        results = pickle.load(f)
        wrong_idx.extend(results['wrong_idx'])
    for i in range(len(results['scores'])):
        idx = 20 * file_idx + i
        if idx == 1:
            print(results['scores'])
        y_label_i = target[idx][0]
        y_labels.append(y_label_i)
        # Calculate the score
        # This is a tuple
        scores = results['scores'][i]
        causal = sum(scores[0])
        anti_causal = sum(scores[1])
        final_score = - causal + anti_causal
        y_scores.append(final_score)
        all_bfgs_scores[idx] = scores 
        all_causal_scores[idx] = y_label_i * final_score

KeyError: 'scores'

In [ ]:
file_idx=0
with open(f"{work_dir}/results/{eval(f'file_{file_idx + 1}')}", "rb") as f:
    results = pickle.load(f)

In [13]:
results

{'correct_idx': [1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19],
 'wrong_idx': [0, 7, 9, 12],
 'weight': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.

In [14]:
all_bfgs_scores[180]

((2071.0507171615855, -488.67826977666937),
 (2047.8954359043792, -673.3959114638874))

In [15]:
all_causal_scores

{0: -12.114390230618937,
 1: 284.5326514661642,
 2: 279.83120313142445,
 3: 17.395608348377664,
 4: 45.93583841598229,
 5: 81.85389654549272,
 6: 44.435692672758705,
 7: -6.421862245766533,
 8: 95.73492144145303,
 9: -48.418618209531815,
 10: 6.957508071929624,
 11: 77.66285510796433,
 12: -26.891980110673103,
 13: 35.189141533139264,
 14: 95.41821323115573,
 15: 0.3408355943638526,
 16: 143.4195481911056,
 17: 218.4393868578777,
 18: 5.054548218216951,
 19: 109.78907114075992,
 20: 28.697208362958918,
 21: 44.262314822859935,
 22: 63.6230415701375,
 23: -58.962321215669135,
 24: -35.6107753167862,
 25: 107.76747841372617,
 26: 35.40911693886028,
 27: 89.02076599027805,
 28: 11.00014171911289,
 29: 50.79858638035694,
 30: -13.024627371521547,
 31: 368.2460493501035,
 32: 35.98769736096983,
 33: 75.30338155099298,
 34: -12.61936242465572,
 35: 25.49257089349294,
 36: 43.28685592235661,
 37: 45.90899259196067,
 38: 130.53170076988317,
 39: -33.39503614159571,
 40: 340.5447087377488,
 41:

In [16]:
all_causal_scores

{0: -12.114390230618937,
 1: 284.5326514661642,
 2: 279.83120313142445,
 3: 17.395608348377664,
 4: 45.93583841598229,
 5: 81.85389654549272,
 6: 44.435692672758705,
 7: -6.421862245766533,
 8: 95.73492144145303,
 9: -48.418618209531815,
 10: 6.957508071929624,
 11: 77.66285510796433,
 12: -26.891980110673103,
 13: 35.189141533139264,
 14: 95.41821323115573,
 15: 0.3408355943638526,
 16: 143.4195481911056,
 17: 218.4393868578777,
 18: 5.054548218216951,
 19: 109.78907114075992,
 20: 28.697208362958918,
 21: 44.262314822859935,
 22: 63.6230415701375,
 23: -58.962321215669135,
 24: -35.6107753167862,
 25: 107.76747841372617,
 26: 35.40911693886028,
 27: 89.02076599027805,
 28: 11.00014171911289,
 29: 50.79858638035694,
 30: -13.024627371521547,
 31: 368.2460493501035,
 32: 35.98769736096983,
 33: 75.30338155099298,
 34: -12.61936242465572,
 35: 25.49257089349294,
 36: 43.28685592235661,
 37: 45.90899259196067,
 38: 130.53170076988317,
 39: -33.39503614159571,
 40: 340.5447087377488,
 41:

In [17]:
len(y_labels)

295

In [18]:
y_labels = np.array(y_labels)
y_scores = np.array(y_scores)

In [25]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc


fpr, tpr, _ = roc_curve(y_labels, y_scores)
roc_auc = auc(fpr, tpr)

In [26]:
roc_auc

0.10535517114464484

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = (roc_auc_score(y_labels == 1, y_scores) + roc_auc_score(y_labels == -1, - y_scores)) / 2
roc_auc

In [ ]:
(roc_auc_score(y_labels == 1, y_scores))

In [ ]:
textstr = 'roc auc=%.3f\n'%(
    roc_auc
)
plt.text(0.8, 0.2, textstr, fontsize=10)
plt.plot(
    fpr, tpr
)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.show()

In [ ]:
all_causal_scores

In [ ]:
all_bfgs_scores

In [ ]:
wrong_idx

In [ ]:
for i in wrong_idx:
    print(i)
    input, output = x[i], y[i]
    plt.scatter(input[:, 0], output[:, 0])
    plt.show()
    plt.close()

# Adam results

In [1]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")

In [2]:
data_name = "cha"

In [3]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np



work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
file_1 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:0_end:20.p"
file_2 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:20_end:40.p"
file_3 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:40_end:60.p"
file_4 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:60_end:80.p"
file_5 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:80_end:100.p"
file_6 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:100_end:120.p"
file_7 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:120_end:140.p"
file_8 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:140_end:160.p"
file_9 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:160_end:180.p"
file_10 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:180_end:200.p"
file_11 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:200_end:220.p"
file_12 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:220_end:240.p"
file_13 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:240_end:260.p"
file_14 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:260_end:280.p"
file_15 = f"fullscore-{data_name}_pairs-gplvm_adam-reinit2-numind200_start:280_end:300.p"

In [17]:
import importlib
from data import get_data

data_get = getattr(get_data, f"get_{data_name}_pairs_dataset")

x, y, weight, target = data_get(data_path=f"{work_dir}/data/{data_name}_pairs/files")

In [18]:
all_bfgs_cha_scores

NameError: name 'all_bfgs_cha_scores' is not defined

In [19]:
y_labels = []
y_scores = []
wrong_idx = []
all_scores = {}
all_causal_scores = {}

for file_idx in range(15):
    with open(f"{work_dir}/results/{eval(f'file_{file_idx + 1}')}", "rb") as f:
        results = pickle.load(f)
        # wrong_idx.extend(results['wrong_idx'])
    for i in range(len(results['scores'])):
        idx = 20 * file_idx + i
        y_label_i = target[idx][0]
        y_labels.append(y_label_i)
        # Calculate the score
        # This is a tuple
        scores = results['scores'][i]
        # bfgs_score = all_bfgs_scores[idx]
        bfgs_score = all_bfgs_cha_scores[idx]
        print(f"Run: {idx}", bfgs_score, scores, '\n')
        causal = min(scores[0][0], bfgs_score[0][0]) + min(scores[0][1], bfgs_score[0][1]) 
        anti_causal =  min(scores[1][0], bfgs_score[1][0]) + min(scores[1][1], bfgs_score[1][1]) 
        # causal = sum(scores[0])
        # anti_causal = sum(scores[1])
        final_score = - causal + anti_causal
        y_scores.append(final_score)
        all_scores[idx] = scores 
        all_causal_scores[idx] = y_label_i * final_score
        if y_label_i * final_score < 0: 
            wrong_idx.append(idx)

NameError: name 'all_bfgs_cha_scores' is not defined

In [ ]:
all_causal_scores

In [ ]:
target[244]

In [ ]:
all_bfgs_scores[244]

In [ ]:
all_scores[244]

In [ ]:
y_labels = np.array(y_labels)
y_scores = np.array(y_scores)

In [ ]:
print(len(y_scores))

In [ ]:
target[244]

In [ ]:
wrong_idx

In [ ]:
wrong = []
for i in range(len(y_labels)):
    if y_labels[i] < 0 and y_scores[i] > 0:
        wrong.append(i)
    elif y_labels[i] > 0 and y_scores[i] < 0:
        wrong.append(i)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc


fpr, tpr, _ = roc_curve(y_labels, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

In [ ]:
len(wrong)

In [ ]:
all_causal_scores

In [ ]:
textstr = 'roc auc=%.3f\n'%(
    roc_auc
)
plt.text(0.8, 0.2, textstr, fontsize=10)
plt.plot(
    fpr, tpr
)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.show()

In [ ]:
all_scores

In [ ]:
all_causal_scores

In [ ]:
for i in wrong_idx:
    print(i)
    input, output = x[i], y[i]
    plt.scatter(input[:, 0], output[:, 0])
    plt.show()
    plt.close()

In [ ]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")

In [ ]:
import pickle
import numpy as np

with open('/vol/bitbucket/ad6013/Research/gp-causal/results/fullscore-cep-gplvm-sqexp-reinit20.p', 'rb') as f:
    results = pickle.load(f)

In [ ]:
results["wrong_idx"]

In [ ]:
res = results['scores']

In [ ]:
res[40]

In [ ]:
sum(res[0])

In [ ]:
y_scores = np.zeros(len(res))
for i in range(len(res)):
    causal_score = - sum(res[i][0]) + sum(res[i][1])
    y_scores[i] = causal_score

In [ ]:
y_scores[0:20] = -1 * y_scores[0:20] 

In [ ]:
y_labels = np.ones(43)
y_labels[0:20] = -1 * y_labels[0:20]

In [ ]:
y_labels

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc


fpr, tpr, _ = roc_curve(y_labels, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
roc_auc

# CEP Results

In [4]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")

In [5]:
data_name = "cep"

In [6]:
import dill 
import matplotlib.pyplot as plt
import numpy as np



work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"
file_1 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:0_end:5.p"
file_2 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:5_end:10.p"
file_3 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:10_end:15.p"
file_4 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:15_end:20.p"
file_5 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:20_end:25.p"
file_6 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:25_end:30.p"
file_7 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:30_end:35.p"
file_8 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:35_end:40.p"
file_9 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:40_end:45.p"
file_10 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:45_end:50.p"
file_11 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:50_end:55.p"
file_12 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:55_end:60.p"
file_13 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:60_end:65.p"
file_14 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:65_end:70.p"
file_15 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:70_end:75.p"
file_16 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:75_end:80.p"
file_17 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:80_end:85.p"
file_18 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:85_end:90.p"
file_19 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:90_end:95.p"
file_20 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:95_end:100.p"
file_21 = f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:100_end:105.p"

In [7]:

new_file_1 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:0_end:5.p"
new_file_2 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:5_end:10.p"
new_file_3 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:10_end:15.p"
new_file_4 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:15_end:20.p"
new_file_5 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:20_end:25.p"
new_file_6 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:25_end:30.p"
new_file_7 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:30_end:35.p"
new_file_8 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:35_end:40.p"
new_file_9 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:40_end:45.p"
new_file_10 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:45_end:50.p"
new_file_11 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:50_end:55.p"
new_file_12 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:55_end:60.p"
new_file_13 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:60_end:65.p"
new_file_14 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:65_end:70.p"
new_file_15 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:70_end:75.p"
new_file_16 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:75_end:80.p"
new_file_17 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:80_end:85.p"
new_file_18 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:85_end:90.p"
new_file_19 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:90_end:95.p"
new_file_20 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:95_end:100.p"
new_file_21 = f"fullscore-{data_name}-gplvmgeneralised-reinit20-numind200_start:100_end:105.p"

In [8]:
import importlib
from data import get_data

data_get = getattr(get_data, f"get_tubingen_pairs_dataset")

x, y, weight, target = data_get(data_path=f"{work_dir}/data/pairs/files")

Load cause-effect pairs: 100%|██████████| 108/108 [00:00<00:00, 260.92it/s]


In [9]:
# import shutil


# for file_idx in range(21):
#     shutil.copy(f"{work_dir}/results/{eval(f'file_{file_idx + 1}')}", f"{work_dir}/results/{eval(f'new_file_{file_idx + 1}')}")

In [10]:
all_results = {}
for file_idx in range(21):
    with open(f"{work_dir}/results/{eval(f'file_{file_idx + 1}')}", "rb") as f:
        results = dill.load(f)
    all_results.update(results["final_scores"])

2022-12-06 12:19:55.678702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 12:19:55.841645: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-06 12:19:58.021179: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /vol/cuda/11.2.1-cudnn8.1.0.77/targets/x86_64-linux/lib:/vol/cuda/11.2.1-cudnn8.1.0.77/lib64:
2022-12-06 12:19:58.021654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic lib

In [11]:
from utils import return_all_scores, return_best_causal_scores, get_correct


all_x, all_y_x, all_y, all_x_y = return_all_scores(all_results)
best_scores = return_best_causal_scores(all_x, all_y_x, all_y, all_x_y)

In [12]:
len(all_x.keys())

99

In [13]:
def get_scores(best_scores):
    total_runs = len(list(best_scores.keys()))
    y_scores = {}
    y_scores_array = np.zeros(total_runs)
    for idx, run_idx in enumerate(list(best_scores.keys())):
        current_run = best_scores[run_idx]
        causal_score = current_run.best_loss_x + current_run.best_loss_y_x
        anticausal_score = current_run.best_loss_y + current_run.best_loss_x_y
        y_scores[run_idx] = - causal_score + anticausal_score
        y_scores_array[idx] = - causal_score + anticausal_score 
    return y_scores, y_scores_array

In [14]:
from tqdm import trange
from sklearn.metrics import roc_auc_score


auc_all = []
for i in trange(1000):
    _, y_scores = get_scores(best_scores)
    total_runs = len(y_scores)

    target = np.ones(total_runs)
    flip_idx = np.random.choice(np.arange(total_runs), total_runs // 2, replace=False)
    for i in range(total_runs):
        if i in flip_idx:
            target[i] *= -1
            y_scores[i] *= -1
    roc_auc = roc_auc_score( target, y_scores )
    auc_all.append(roc_auc)

100%|██████████| 1000/1000 [00:00<00:00, 1402.76it/s]


In [15]:
auc_all

[0.786530612244898,
 0.7816326530612245,
 0.7832653061224489,
 0.7812244897959184,
 0.7861224489795918,
 0.783265306122449,
 0.7808163265306123,
 0.7877551020408163,
 0.7808163265306123,
 0.7824489795918367,
 0.7857142857142857,
 0.7848979591836734,
 0.7844897959183673,
 0.7869387755102042,
 0.7857142857142857,
 0.7836734693877552,
 0.7857142857142857,
 0.7840816326530612,
 0.7861224489795918,
 0.7836734693877551,
 0.7820408163265306,
 0.7840816326530612,
 0.7881632653061225,
 0.7828571428571428,
 0.7808163265306123,
 0.7808163265306123,
 0.7848979591836734,
 0.786530612244898,
 0.7824489795918368,
 0.7738775510204081,
 0.7812244897959184,
 0.789795918367347,
 0.7783673469387755,
 0.7824489795918368,
 0.7783673469387755,
 0.7914285714285714,
 0.7812244897959184,
 0.7693877551020408,
 0.7808163265306123,
 0.7804081632653062,
 0.7836734693877551,
 0.7869387755102041,
 0.7861224489795918,
 0.7820408163265307,
 0.7873469387755102,
 0.7820408163265307,
 0.7795918367346939,
 0.78612244897959

In [16]:
np.mean(auc_all)


0.7829628571428572

In [17]:
np.mean(auc_all)

0.7829628571428572

In [18]:
# Get the weighted accuracy
y_dict, y_scores = get_scores(best_scores)
correct_weight = []
wrong_weight = []
for run_idx in list(y_dict.keys()):
    curr_score = y_dict[run_idx]
    if curr_score < 0:
        print(run_idx)
        wrong_weight.append(weight[run_idx])
    if curr_score > 0:
        correct_weight.append(weight[run_idx])

2
12
13
14
15
17
20
21
24
39
44
45
55
56
57
58
59
60
62
65
66
72
73
76
78
89
90
92
97
99


In [19]:
# Find weighted accuracy
correct_total_weight = np.sum(correct_weight)
wrong_total_weight = np.sum(wrong_weight)
accuracy = correct_total_weight / (correct_total_weight + wrong_total_weight)
print(accuracy)

0.7607943005079173


In [ ]:
fpr, tpr, _ = roc_curve(-1 * target, -1 * y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
print(roc_auc)

In [ ]:
random_choice = np.random.choice(len(y_labels), size=len(y_labels) // 2)
for i in random_choice:
    y_labels[i] *= -1
    y_scores[i] *= -1

In [ ]:
random_choice

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_curve, auc


fpr, tpr, _ = roc_curve(y_labels, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_labels, y_scores)
print(auc)

 # GPLVM Data Results

In [1]:
import sys
sys.path.append("/vol/bitbucket/ad6013/Research/gp-causal")

In [2]:
import dill 
import matplotlib.pyplot as plt
import numpy as np
work_dir = "/vol/bitbucket/ad6013/Research/gp-causal"


def get_files(data_name):
    all_files = [
        f"fullscore-{data_name}-gplvmgeneralised-reinit10-numind200_start:{i}_end:{i + 10}.p" for i in np.linspace(0, 90, 10, dtype=int)
    ]
    return all_files

In [3]:
def get_result_files(all_files):
    all_results = {}
    for file in all_files:
        with open(f"{work_dir}/results/{file}", "rb") as f:
            results = dill.load(f)
        all_results.update(results["final_scores"])
    return all_results

In [4]:
all_files = get_files("gplvm_pairs")
all_results = get_result_files(all_files)

FileNotFoundError: [Errno 2] No such file or directory: '/vol/bitbucket/ad6013/Research/gp-causal/results/fullscore-gplvm_pairs-gplvmgeneralised-reinit10-numind200_start:0_end:10.p'

In [ ]:
from utils import return_all_scores, return_best_causal_scores, get_correct


all_x, all_y_x, all_y, all_x_y = return_all_scores(all_results)
best_scores = return_best_causal_scores(all_x, all_y_x, all_y, all_x_y)

In [ ]:
total_runs = len(list(best_scores.keys()))
y_scores_gplvm = np.zeros(total_runs)
for idx, run_idx in enumerate(list(best_scores.keys())):
    current_run = best_scores[run_idx]
    causal_score = current_run.best_loss_x + current_run.best_loss_y_x
    anticausal_score = current_run.best_loss_y + current_run.best_loss_x_y
    y_scores_gplvm[idx] = causal_score - anticausal_score 

In [ ]:
y_scores_linear

In [ ]:
plt.bar(np.arange(total_runs), y_scores_linear, alpha=0.8)
plt.bar(np.arange(-50, 0), y_scores_gplvm[:50], alpha=0.8)
plt.bar(np.arange(100, 150), y_scores_gplvm[50:], alpha=0.8)
plt.show()
plt.close()

In [ ]:
y_scores

In [ ]:
np.max(y_scores)

In [ ]:
correct_idx, wrong_idx = get_correct(best_scores, target)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(final_targets, y_scores)
roc_auc = auc(fpr, tpr)

In [ ]:
print(roc_auc)

In [ ]:
y_scores